In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import nn
from tensorflow.keras.activations import softmax
from keras import layers
from tensorflow.keras.layers import Dense,LayerNormalization ## alternative for nn.linear
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

2023-12-05 08:45:34.773633: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-05 08:45:34.874480: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 08:45:34.874561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 08:45:34.877091: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 08:45:34.897416: I tensorflow/core/platform/cpu_feature_guar

In [20]:
X_data = np.load("../../data/data_64_30.npy")
X_data = np.transpose(X_data, (0, 2, 1))
print(X_data.shape)
print(X_data[0,:,1])

(13100, 30, 64)
[  72.01083    147.46912    106.03668    131.80023     12.418795
  151.86609    118.946655   122.67044    114.2478     125.68599
  -29.96453     85.215195    47.004295   -87.80414     68.71507
   86.04312    -34.267963    45.795727   160.45639     96.857025
  135.44983    -71.43028     38.807117    -7.3336506  167.19531
 -101.842926     0.           0.           0.           0.       ]


In [3]:
data = pd.read_csv(
    "../../data/LJSpeech-1.1/metadata.csv",
    sep="|",
    header=None,
    names=["ID", "Text1", "Text2"],
)
texts = data["Text1"].to_list()
ID = data["ID"].to_list()
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
num_classes = len(tokenizer.word_index) + 1  # Add 1 for the padding token
sequences = tokenizer.texts_to_sequences(texts)
Y_data = pad_sequences(sequences, padding="post", maxlen=30)
# Y_data=Y_data[:100]
print(num_classes)
print(Y_data.shape)

14518
(13100, 30)


In [4]:
def create_self_attention_mask(sequence_length):
    mask = np.tril(np.ones((sequence_length,sequence_length)))
    mask[mask==0]=-np.inf
    mask[mask==1]=0
    return mask

In [5]:
test = create_self_attention_mask(5)
test

array([[  0., -inf, -inf, -inf, -inf],
       [  0.,   0., -inf, -inf, -inf],
       [  0.,   0.,   0., -inf, -inf],
       [  0.,   0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.,   0.]])

In [6]:
def scaled_dot_product(q, k, v, mask):
    d_k = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_qk = tf.matmul(q, k, transpose_b=True) / tf.math.sqrt(d_k)

    if mask is not None:
        scaled_qk += mask

    attention_weights = tf.nn.softmax(scaled_qk)
    output = tf.matmul(attention_weights, v)
    return output, attention_weights

In [7]:
q=[[1.0,2.0,1.0],[1.0,1.0,1.0],[1.0,1.0,1.0]]
k=[[1.0,3.0,1.0],[1.0,1.0,1.0],[1.0,1.0,1.0]]
v=[[1.0,1.0,1.0],[1.0,1.0,1.0],[1.0,1.0,1.0]]
mask_1=create_self_attention_mask(3)
test,weights = scaled_dot_product(q,k,v,mask=mask_1)
print("output is\n",test)
print("attention weights is \n",weights)

output is
 tf.Tensor(
[[1.         1.         1.        ]
 [0.99999994 0.99999994 0.99999994]
 [1.         1.         1.        ]], shape=(3, 3), dtype=float32)
attention weights is 
 tf.Tensor(
[[1.         0.         0.        ]
 [0.7603684  0.23963155 0.        ]
 [0.6133826  0.19330868 0.19330868]], shape=(3, 3), dtype=float32)


2023-12-05 08:45:41.054280: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-12-05 08:45:41.054336: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: Pc
2023-12-05 08:45:41.054350: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: Pc
2023-12-05 08:45:41.054551: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2023-12-05 08:45:41.054578: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2023-12-05 08:45:41.054585: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [8]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, d_model, max_sequence_length):
        super(PositionalEncoding, self).__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    
    def call(self, inputs):
        even_i = tf.range(0, self.d_model, 2, dtype=tf.float32)
        denominator = tf.pow(10000.0, even_i / self.d_model)
        position = tf.reshape(
            tf.range(self.max_sequence_length, dtype=tf.float32),
            (1, self.max_sequence_length, 1),
        )
        even_PE = tf.sin(position / denominator)
        odd_PE = tf.cos(position / denominator)
        stacked = tf.stack([even_PE, odd_PE], axis=2)
        PE = tf.reshape(stacked, (1, self.max_sequence_length, -1))
        print("postional encoding output shape",PE.shape)
        return PE

In [9]:
class ConvolutionalLayer(tf.keras.layers.Layer):
    def __init__(self, input_shape, filters=64, kernel_size=3, **kwargs):
        super(ConvolutionalLayer, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size

        self.conv1 = layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, padding="same", trainable=True)
        self.batch_norm1 = layers.BatchNormalization()
        self.relu1 = layers.ReLU()

        self.conv2 = layers.Conv1D(filters=self.filters, kernel_size=self.kernel_size, padding="same",trainable=True)
        self.batch_norm2 = layers.BatchNormalization()
        self.relu2 = layers.ReLU()

        self.global_avg_pooling = layers.GlobalAveragePooling1D()

    
    def call(self, inputs, training=None, mask=None):
        conv1_out = self.relu1(self.batch_norm1(self.conv1(inputs), training=training))
        conv2_out = self.relu2(self.batch_norm2(self.conv2(conv1_out), training=training))
        gap_out = self.global_avg_pooling(conv2_out)
        if training:
            print("CNN output shape is  ", conv2_out.shape)
        return conv2_out

In [10]:
class CNN_model(tf.keras.Model):
    def __init__(self, d_model, max_input_length):
        super(CNN_model, self).__init__()
        self.d_model = d_model
        self.cnn_layer = ConvolutionalLayer(input_shape=(None, 30,64))  # Adjust input shape

    def call(self, inputs, training=None):
        cnn_output = self.cnn_layer(inputs)
        return cnn_output


In [11]:
model = tf.keras.Sequential([CNN_model(d_model=20,max_input_length=30),layers.Dense(30)])
# model.build(input_shape=(None, 500, 20))

In [12]:
from sklearn.model_selection import train_test_split

# Assuming X_data and Y_data are your input features and labels
X_train, X_val, Y_train, Y_val = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_val.shape)

print(Y_train.shape)
print(Y_val.shape)

(10480, 30, 64)
(2620, 30, 64)
(10480, 30)
(2620, 30)


In [13]:
optim = tf.keras.optimizers.Adam(learning_rate=5)
model.compile(optimizer=optim, loss='mean_squared_error',metrics=['mae'])
model.build(input_shape=(13100,30,64))  # Replace your_input_shape with the actual input shape
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cnn_model (CNN_model)       (13100, 30, 64)           25216     
                                                                 
 dense (Dense)               (13100, 30, 30)           1950      
                                                                 
Total params: 27166 (106.12 KB)
Trainable params: 26910 (105.12 KB)
Non-trainable params: 256 (1.00 KB)
_________________________________________________________________


In [14]:
def loss_function(y_true, y_pred):
    return tf.reduce_mean(tf.square(y_true - y_pred))

In [15]:
model.fit(X_train,Y_train,epochs=1,validation_data=(X_val, Y_val))

CNN output shape is   (None, 30, 64)
CNN output shape is   (None, 30, 64)
CNN output shape is   (None, 30, 64)


InvalidArgumentError: Graph execution error:

Detected at node gradient_tape/mean_squared_error/BroadcastGradientArgs defined at (most recent call last):
  File "/home/abhi/anaconda3/envs/test/lib/python3.9/runpy.py", line 197, in _run_module_as_main

  File "/home/abhi/anaconda3/envs/test/lib/python3.9/runpy.py", line 87, in _run_code

  File "/home/abhi/anaconda3/envs/test/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/abhi/anaconda3/envs/test/lib/python3.9/site-packages/traitlets/config/application.py", line 1077, in launch_instance

  File "/home/abhi/anaconda3/envs/test/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 737, in start

  File "/home/abhi/anaconda3/envs/test/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 199, in start

  File "/home/abhi/anaconda3/envs/test/lib/python3.9/asyncio/base_events.py", line 601, in run_forever

  File "/home/abhi/anaconda3/envs/test/lib/python3.9/asyncio/base_events.py", line 

In [ ]:
prediction = model.predict(X_val)
print("prediction ",prediction[0])
print("actual ",Y_val[0])